# 任务目标--使用TF-IDF对api_sequence提取特征

# 调包区


In [1]:
from tqdm.notebook import tqdm
import random
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import gc

# 警告忽略
import warnings
warnings.filterwarnings("ignore")

# matplotlib字体设置
plt.rcParams["font.family"] = "Songti SC"
plt.rcParams["axes.unicode_minus"] = False

# matplotlib警告忽略
pd.plotting.register_matplotlib_converters()


# 观看Dataframe长度
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
# 浮点数位长度
pd.set_option('display.precision',5)

# 显示多个结果
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' # ['all', 'last', 'last_expr', 'none', 'last_expr_or_assign']

In [2]:
from sklearn.preprocessing import StandardScaler,LabelEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold,KFold
from sklearn.naive_bayes import BernoulliNB,GaussianNB,MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from lightgbm.sklearn import LGBMClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score,recall_score,precision_score,roc_auc_score,auc,f1_score
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_squared_log_error
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
import keras
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D , GlobalAveragePooling1D,MaxPool1D
from keras import Model
from keras.models import Sequential
from keras.optimizers import Adam,SGD # 优化方法
from keras.callbacks import EarlyStopping,ModelCheckpoint,RemoteMonitor,CSVLogger
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [4]:
import networkx as nx
import node2vec
from node2vec import Node2Vec
from gensim.models import word2vec

# 打开文件

In [5]:
#path = './sampledata' # 打开样本数据
path = './fulldata' # 打开全量数据

train_data = pd.read_csv(f'{path}/security_train.csv')
test_data = pd.read_csv(f'{path}/security_test.csv')

test_data['label']=np.nan

# 特征工程

## 生成特征文档

In [6]:
basic_feature = ['file_id','label','tid','index']

train_fe = train_data[basic_feature].drop_duplicates(['file_id','label'])
test_fe = test_data[basic_feature].drop_duplicates(['file_id','label'])


## 构建统计值特征（count,nunique,min、max、sum、mean、median）

In [7]:
# 得到统计值特征（count,nunique,min、max、sum、mean、median）
def creat_stats_feature(tem_df):
    # 以file_id 为单位聚合，统计每一个file_id下的情况
    group_df = tem_df.groupby('file_id')

    # 得到 count,nunique,min、max、sum、mean、median的统计值
    stats_dict = {
        "tid":['count', 'nunique', 'min', 'max', 'mean', 'median','std'], 
        "index":['count', 'nunique', 'min', 'max', 'mean', 'median','std'],}

    stats_df = group_df.agg(stats_dict)
    stats_df.columns = ["_".join(tup) for tup in stats_df.columns]
    stats_df = stats_df.reset_index()
    return(stats_df)



# 构建训练集的特征
stats_df = creat_stats_feature(train_data)
train_fe = train_fe.merge(stats_df,on='file_id',how='left')

# 构建测试集的特征
stats_df = creat_stats_feature(test_data)
test_fe = test_fe.merge(stats_df,on='file_id',how='left')


# 回收内存
del stats_df
gc.collect()


3

## 构建TFIDF特征

In [8]:
%%time
############################################################################################
# 构建api句子
def get_apis(df):
    # 以file_id进行聚合
    group_df = df.groupby('file_id')
    
    # 统计每一个file_id下的api的情况
    file_api = {}
    for file_id,file_group in group_df:
        # 以tid、index从小达到的排序，生成每一个file_id下的api句子
        result = file_group.sort_values(['tid','index'],ascending=True)
        api_sequence = ' '.join(result['api'])
        file_api[file_id] = api_sequence
    return(file_api)
        

## 构建TFIDF特征
def get_tfidf(tem_df):
    tfidf = TfidfVectorizer(ngram_range=(1,3),min_df=0.1,max_df=0.8)
    api_feature = tfidf.fit_transform(tem_df['api'])

    # 转化为DataFrame
    df_apis = pd.DataFrame(api_feature.toarray(),columns=tfidf.get_feature_names()).reset_index(drop=True)
    return(df_apis)

############################################################################################

#### 构建api句子
def creat_tfidf_feature(train_data,test_data,train_fe,test_fe,cache=False):
    # 判断是否需要重新构建特征
    if cache==True:
        path = './temdata'
        df_fe = pd.read_pickle(f'{path}/tfidf_feature.pkl')
        
    
    else:
    ##########################    
        # 构建训练集
        train_api = get_apis(df = train_data)
        temp = pd.DataFrame.from_dict(train_api,orient='index',columns=['api'])
        temp = temp.reset_index().rename(columns={'index':'file_id'})
        train_fe = train_fe.merge(temp,on='file_id',how='left')

        # 构建测试集
        test_api = get_apis(df = test_data)
        temp = pd.DataFrame.from_dict(test_api,orient='index',columns=['api'])
        temp = temp.reset_index().rename(columns={'index':'file_id'})
        test_fe = test_fe.merge(temp,on='file_id',how='left')

        #### 构建tfdif特征
        df_fe = pd.concat([train_fe,test_fe],axis=0).reset_index(drop=True)

        # 得到tfidf特征
        tfidf_feature = get_tfidf(df_fe)


        # 合并tfidf特征
        df_fe = pd.concat([df_fe,tfidf_feature],axis=1).drop(['api'],axis=1)
        
        # 保存tfidf特征
        path = './temdata'
        df_fe.to_pickle(f'{path}/tfidf_feature.pkl')
        
    
    ##########################


    # 划分训练集
    train_fe = df_fe[df_fe['label'].notnull()]

    # 划分测试集
    test_fe = df_fe[df_fe['label'].isnull()]

    # 回收内存
    del tfidf_feature,df_fe
    gc.collect()
    
    return(train_fe,test_fe)




CPU times: user 1min 6s, sys: 22.1 s, total: 1min 28s
Wall time: 1min 33s


,file_id,label,tid,index,tid_count,tid_nunique,tid_min,tid_max,tid_mean,tid_median,tid_std,index_count,index_nunique,index_min,index_max,index_mean,index_median,index_std,api
0,1,NaN,2332,0,97,4,2332,2568,2518.63918,2544,57.21855,97,31,0,30,14.44330,14.0,9.21047,RegOpenKeyExA CopyFileA OpenSCManagerA CreateS...
1,2,NaN,2472,0,1361,7,2472,2748,2576.86407,2524,104.39915,1361,681,0,680,339.75018,340.0,196.51574,RegOpenKeyExA RegOpenKeyExA RegOpenKeyExA NtCl...
2,3,NaN,2344,0,16,1,2344,2344,2344.00000,2344,0.00000,16,16,0,15,7.50000,7.5,4.76095,GetSystemTimeAsFileTime SetUnhandledExceptionF...
3,4,NaN,2452,0,193,3,2452,2584,2476.33161,2452,50.95151,193,193,0,192,96.00000,96.0,55.85845,LdrLoadDll LdrGetProcedureAddress CreateToolhe...
4,5,NaN,2332,0,803,3,2332,2780,2496.14944,2376,201.82681,803,268,0,267,133.33375,133.0,77.31705,GetSystemTimeAsFileTime NtAllocateVirtualMemor...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12950,12951,NaN,2644,0,289,6,2644,2980,2716.53979,2776,75.40253,289,145,0,144,71.75087,72.0,41.78641,NtAllocateVirtualMemory NtFreeVirtualMemory Nt...
12951,12952,NaN,2264,0,112,1,2264,2264,2264.00000,2264,0.00000,112,112,0,111,55.50000,55.5,32.47563,NtAllocateVirtualMemory NtFreeVirtualMemory Nt...
12952,12953,NaN,2324,0,5095,15,2324,2884,2622.11816,2708,196.69573,5095,1464,0,1463,538.42375,454.0,393.60502,LdrLoadDll LdrGetProcedureAddress LdrGetProced...
12953,12954,NaN,2424,0,2951,4,2424,2700,2593.64690,2680,126.12415,2951,1445,0,1444,596.70180,555.0,397.35807,LdrLoadDll LdrGetProcedureAddress LdrGetProced...


# 特征工程保存

In [10]:
path = "./temdata"
train_fe.to_pickle(f'{path}/train_fe_v1.pkl')
test_fe.to_pickle(f'{path}/test_fe_v1.pkl')

In [11]:
path = "./temdata"
train_fe = pd.read_pickle(f'{path}/train_fe_v1.pkl')
test_fe = pd.read_pickle(f'{path}/test_fe_v1.pkl')

# 数据切分

In [12]:
# 数据划分
raw_x,raw_y = train_fe.drop('label',axis=1),train_fe['label']
sub_x,sub_y = test_fe.drop('label',axis=1),test_fe['label']

# 进行数据切分
train_x,vaild_x,train_y,vaild_y = train_test_split(raw_x,raw_y,test_size=0.2)

# 建模尝试

## 使用lgb进行建模预测

In [13]:
# 参数设置
gpu=False

#############################################

if gpu==True:
    params = {
    'device': 'gpu',
    'gpu_platform_id': 0,
    'gpu_device_id': 0}
else:
    params = {}




#############################################

model = lgb.LGBMClassifier(
            num_leaves=2**5-1, reg_alpha=0.25, reg_lambda=0.25, objective='multiclass',
            max_depth=-1, learning_rate=0.005, min_child_samples=3, random_state=2021,
            n_estimators=2000, subsample=1, colsample_bytree=1,**params)


#############################################
# 模型训练
# model.fit(
#     train_x, train_y,
#     eval_metric='logloss', eval_set=[(train_x, train_y), (vaild_x, vaild_y)],
#     verbose=100,
#     #早停法，如果auc在10epoch没有进步就stop
#     early_stopping_rounds=1000 )

model.fit(raw_x,raw_y)


# 预测
pred_prob = model.predict_proba(sub_x)


#############################################
# 转化为提交文件
# 转化为DataFrame
sub_df = pd.DataFrame(pred_prob)
sub_df.insert(0,"file_id",sub_x['file_id'].reset_index(drop=True))

# 更改列名
sub_df.columns = list(map(lambda x:f"prob{x}" if x!='file_id' else x,sub_df.columns))
sub_df = sub_df.astype('double')
sub_df['file_id'] = sub_df['file_id'].astype(int)

# 保存在本地
path = './outdata'
sub_df.to_csv(f'{path}/lgb_v1.csv',index=None)

#############################################
# 保存模型
import joblib
# 模型存储
joblib.dump(model, './modelfile/lgb_v1.pkl')
# 模型加载
# model = joblib.load('lgb.pkl')
print('模型保存完毕！')

LGBMClassifier(colsample_bytree=1, learning_rate=0.005, min_child_samples=3,
               n_estimators=2000, objective='multiclass', random_state=2021,
               reg_alpha=0.25, reg_lambda=0.25, subsample=1)

['./modelfile/lgb_v1.pkl']

模型保存完毕！


## 使用xgb进行建模预测

In [14]:
# 参数设置
gpu=False

#############################################

if gpu==True:
    params = {'tree_method': 'gpu_hist'}

else:
    params = {}



#############################################

model = xgb.XGBClassifier(
            max_depth=9, learning_rate=0.005, n_estimators=2000, 
            objective='multi:softprob', 
            subsample=0.8, colsample_bytree=0.8, 
            min_child_samples=3, eval_metric='mlogloss', reg_lambda=0.5,**params)


#############################################
# 模型训练
# model.fit(
#     train_x, train_y,
#     eval_metric='mlogloss', eval_set=[(train_x, train_y), (vaild_x, vaild_y)],
#     verbose=100,
#     #早停法，如果auc在10epoch没有进步就stop
#     early_stopping_rounds=1000 )


model.fit(raw_x,raw_y)

# 预测
pred_prob = model.predict_proba(sub_x)


#############################################
# 转化为提交文件
# 转化为DataFrame
sub_df = pd.DataFrame(pred_prob)
sub_df.insert(0,"file_id",sub_x['file_id'].reset_index(drop=True))

# 更改列名
sub_df.columns = list(map(lambda x:f"prob{x}" if x!='file_id' else x,sub_df.columns))

# 保存在本地
path = './outdata'
sub_df = sub_df.astype('double')
sub_df['file_id'] = sub_df['file_id'].astype(int)
sub_df.to_csv(f'{path}/xgb_v1.csv',index=None)


#############################################
# 保存模型
import joblib
# 模型存储
joblib.dump(model, './modelfile/xgb_v1.pkl')
# 模型加载
# model = joblib.load('xgb.pkl')
print('模型保存完毕！')

[14:06:17] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { min_child_samples } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, eval_metric='mlogloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.005, max_delta_step=0,
              max_depth=9, min_child_samples=3, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=2000, n_jobs=0,
              num_parallel_tree=1, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=0.5, scale_pos_weight=None, subsample=0.8,
              tree_method='exact', validate_parameters=1, verbosity=None)

['./modelfile/xgb_v1.pkl']

模型保存完毕！


# 进行模型融合

## 普通的加权融合

In [15]:
# 打开之前的预测结果proba
path = './outdata'
lgb_pred = pd.read_csv(f'{path}/lgb_v1.csv').set_index('file_id')
xgb_pred = pd.read_csv(f'{path}/xgb_v1.csv').set_index('file_id')

# 设置权重
lgb_weight = 0.4
xgb_weight = 0.6

# 进行加权融合
mix_pred = (lgb_weight*lgb_pred)+(xgb_weight*xgb_pred)

# 保存加权融合后的结果
mix_pred.to_csv(f'{path}/mix_v1.csv')

In [16]:
raw_x

file_id   tid  index  tid_count  tid_nunique  tid_min  tid_max  \
0            1  2488      0       6786           11     2488     2812   
1            2  2320      0        816            4     2320     2604   
2            3  2208      0        463            1     2208     2208   
3            4  2284      0       2046            9     2284     2980   
4            5  2500      0      10002            6     2500     2676   
...        ...   ...    ...        ...          ...      ...      ...   
13882    13883   100      0     178221          998      100     6568   
13883    13884  2592      0       1319            2     2592     2748   
13884    13885  2240      0       1033            3     2240     2744   
13885    13886  2324      0       5316           10     2324     2836   
13886    13887  2336      0        623            1     2336     2336   

         tid_mean  tid_median     tid_std  index_count  index_nunique  \
0      2537.98762        2488    83.88130         6786           5001   
1      2471.00000        2480   101.50678          816            204   
2      2208.00000        2208     0.00000          463            463   
3      2430.72727        2340   150.46051         2046           1028   
4      2549.73125        2596    49.55630        10002           5001   
...           ...         ...         ...          ...            ...   
13882  3544.76290        3392  1405.04552       178221           5001   
13883  2592.11827        2592     4.29539         1319           1319   
13884  2242.54405        2240    33.15202         1033           1033   
13885  2483.39955        2600   154.79679         5316           2503   
13886  2336.00000        2336     0.00000          623            623   

       index_min  index_max  index_mean  index_median   index_std  \
0              0       5000  2000.80696        1607.5  1510.69422   
1              0        203   101.50000         101.5    58.92514   
2              0        462   231.00000         231.0   133.80085   
3              0       1027   511.01222         511.0   295.40788   
4              0       5000  2500.00000        2500.0  1443.73649   
...          ...        ...         ...           ...         ...   
13882          0       5000   401.48099          47.0  1008.63604   
13883          0       1318   659.00000         659.0   380.90681   
13884          0       1032   516.00000         516.0   298.34572   
13885          0       2502  1173.05041        1165.5   755.54565   
13886          0        622   311.00000         311.0   179.98889   

       __exception__  __exception__ ntterminateprocess  closesocket  \
0            0.00032                           0.00044      0.00047   
1            0.00000                           0.00000      0.06168   
2            0.00000                           0.00000      0.00000   
3            0.00000                           0.00000      0.00000   
4            0.00000                           0.00000      0.00000   
...              ...                               ...          ...   
13882        0.00000                           0.00000      0.00000   
13883        0.00000                           0.00000      0.00000   
13884        0.00000                           0.00000      0.00000   
13885        0.00217                           0.00000      0.00000   
13886        0.00000                           0.00000      0.00000   

       cocreateinstance  cocreateinstance cocreateinstance  \
0               0.00497                            0.00207   
1               0.00000                            0.00000   
2               0.00000                            0.00000   
3               0.00000                            0.00000   
4               0.00072                            0.00000   
...                 ...                                ...   
13882           0.03553                            0.01201   
13883           0.02435                            0.00435   
13884        